In [21]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import torch

from pathlib import Path
import random
import time
from tqdm import tqdm
import json

import ssl
import urllib.request

ssl._create_default_https_context = ssl._create_unverified_context

SMALL_DATASET = True
data_path = Path(f'visual-taxonomy-{"small" if SMALL_DATASET==True else ""}/')
test_image_path = data_path / 'test_images'
train_image_path = data_path / 'train_images'
print(data_path)

device = "cuda" if torch.cuda.is_available() else "cpu"

BATCH_SIZE = 32

visual-taxonomy-small


In [22]:
df_train = pd.read_csv(data_path / 'train.csv')
df_test = pd.read_csv(data_path / 'test.csv')
print(df_train.describe())
print(df_test.describe())

                 id         len
count    800.000000  800.000000
mean   35923.855000    8.760000
std    20322.458313    1.597056
min      319.000000    5.000000
25%    18224.750000    8.000000
50%    37700.000000   10.000000
75%    53628.500000   10.000000
max    70376.000000   10.000000
                 id
count    200.000000
mean   14470.730000
std     9026.003275
min       86.000000
25%     6983.750000
50%    13839.500000
75%    21583.500000
max    30461.000000


In [23]:
print(df_train.head())
print('---')
print(df_test.head())

      id             Category  len         attr_1         attr_2      attr_3  \
0  15996               Sarees   10            NaN            NaN         NaN   
1  62298  Women Tops & Tunics   10         maroon        regular     regular   
2  22725               Sarees   10  same as saree  temple border  big border   
3  39408        Women Tshirts    8           pink        regular     regular   
4  41847        Women Tshirts    8        default        regular     regular   

      attr_4       attr_5         attr_6           attr_7         attr_8  \
0        NaN          NaN            NaN       zari woven            NaN   
1   stylised       casual          solid            solid  short sleeves   
2  navy blue        daily       jacquard       zari woven          solid   
3    printed  funky print  short sleeves  regular sleeves            NaN   
4    printed   typography   long sleeves  regular sleeves            NaN   

         attr_9 attr_10  
0           NaN     NaN  
1  puff sl

In [24]:
from PIL import Image

In [25]:
'000000.jpg' in os.listdir(train_image_path)

False

In [26]:
train_range = range(len(df_train))
test_range = range(len(df_test))

In [27]:
if SMALL_DATASET == False:
    plt.figure(figsize=(30, 30))
    for i in range(4):
        for j in range(4):
            idx = random.randint(0, len(df_train))
            img = Image.open(train_image_path / f'{idx:06d}.jpg')
            plt.subplot(4, 4, 4 * i + j + 1)

            plt.imshow(img)
            plt.title(df_train.loc[idx, 'Category'])
        
    plt.show()



In [28]:
import torch
import torchvision
from torch import nn
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from torch import optim


import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
from PIL import Image
from tempfile import TemporaryDirectory

In [29]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [30]:
if SMALL_DATASET == False:
    num = 560
    img = Image.open(train_image_path / f'{num:06d}.jpg')
    img_1 = data_transforms['train'](img)
    plt.figure(figsize=(20, 10))
    plt.subplot(1, 2, 1)
    plt.imshow(img)
    plt.subplot(1, 2, 2)
    plt.imshow(img_1.permute(1, 2, 0))
    plt.show()
    print(df_train.loc[num, 'Category'])

In [31]:
df_train['Category'].value_counts()
df_train.fillna('None', inplace=True)

In [32]:
values = [df_train[col].unique() for col in df_train.columns]
print(values)

[array([15996, 62298, 22725, 39408, 41847, 55370, 24305, 55732,  4647,
       60868, 29390, 61269, 62642, 26950, 11350, 53620, 46455, 23663,
       24452, 69159, 48490, 12380, 28621, 42347, 41830, 29304, 63461,
       31661, 66162, 46049, 51052, 57798, 69725, 66461, 36856, 33033,
       43253, 18487, 54289, 68217, 28528, 37298, 32188, 48082, 60510,
       38759, 49055, 69401, 63901, 45394, 58947, 64612, 35981, 16392,
        9574, 14076, 37302, 28051, 40268, 34551, 38375, 50213, 17945,
       53692, 40921, 61294, 34695, 60662, 22994, 28245, 64559, 39328,
       68042, 44829,  6942, 54992,  7141, 56277, 52150, 54311,  4998,
       69257, 29811, 16160, 29900, 57738, 67047, 60724, 55931, 62916,
        3503,  3244, 15657, 54854,  5019, 66098, 39581, 42698, 18293,
       49389, 42616, 43250, 13727,  8441,  2521, 13041, 13174,  7257,
       32677, 26302, 61385, 58794, 59389, 23582, 12791, 50949, 58444,
       42251,   449, 25577, 17175, 61725, 22581,  4764, 25538, 15621,
       33636, 27325

In [33]:
# list(values[1]).index('Men Tshirts')
values[1]

array(['Sarees', 'Women Tops & Tunics', 'Women Tshirts', 'Men Tshirts',
       'Kurtis'], dtype=object)

In [34]:
def convert_attributes_to_tensor(df):
    # df = df.fillna('None')
    attr_list = [list(values[i]).index(df.iloc[i]) for i in range(1, len(df))]
    return torch.tensor(attr_list)

In [35]:
df_train.loc[0]

id               15996
Category        Sarees
len                 10
attr_1            None
attr_2            None
attr_3            None
attr_4            None
attr_5            None
attr_6            None
attr_7      zari woven
attr_8            None
attr_9            None
attr_10           None
Name: 0, dtype: object

In [36]:
convert_attributes_to_tensor(df_train.loc[0])

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [37]:
class CustomDataset(Dataset):

    def __init__(self, 
                root_dir: str,
                root_csv: str,
                file_numbers: [i for i in range(0, 70000)],
                transform=None):

        self.root_dir = root_dir
        self.root_csv = root_csv
        self.transform = transform
        self.file_numbers = file_numbers 


    def __len__(self):
        return len(self.root_csv)

    def __getitem__(self, idx):
        file_number = self.file_numbers[idx]
        img = Image.open(self.root_dir / f'{file_number:06d}.jpg')
        transformed_img = self.transform(img)

        attributes = convert_attributes_to_tensor(self.root_csv.loc[idx])

        return transformed_img, attributes[0]


In [38]:
with open("rand_list_train.json", "r") as f:
    rand_list_train = json.load(f)

print(rand_list_train)

[41966, 26475, 48830, 62057, 44912, 28701, 42722, 3683, 4784, 55206, 69926, 65503, 2879, 30656, 50534, 27187, 43454, 54809, 18929, 29682, 65132, 43117, 41577, 24352, 1242, 4842, 55609, 27224, 25688, 15118, 56869, 22389, 46106, 7251, 2095, 24907, 58808, 24095, 25031, 50405, 68444, 39358, 40633, 8425, 64787, 56343, 51931, 22300, 7580, 5956, 40763, 35859, 61126, 59603, 69777, 65691, 49639, 8344, 36904, 22406, 37091, 63992, 45119, 58433, 21171, 4821, 24694, 24293, 51443, 17811, 42018, 3236, 6575, 956, 68478, 41112, 23500, 32700, 69970, 13933, 56105, 13477, 67396, 56106, 65296, 30728, 64069, 41212, 30948, 33340, 7183, 32794, 14968, 17401, 50905, 64591, 3475, 34236, 62159, 33494, 26690, 25774, 29793, 45318, 61627, 54008, 59918, 65582, 27303, 38835, 23040, 8490, 12013, 47656, 28937, 47832, 1806, 54299, 8880, 14075, 8801, 37927, 22645, 58625, 25185, 31760, 44168, 22458, 5739, 1858, 10728, 16993, 37602, 19047, 58943, 37840, 11994, 21370, 29633, 18926, 31639, 40218, 37267, 42975, 17764, 6484, 68

In [39]:
with open("rand_list_test.json", "r") as f:
    rand_list_test = json.load(f)

print(rand_list_test)

[21156, 23991, 12552, 4822, 28126, 29747, 922, 24626, 13922, 19260, 1178, 20791, 5581, 22880, 1600, 14282, 16991, 21358, 22821, 7758, 12155, 23848, 26181, 24837, 17091, 18375, 23837, 14808, 29703, 3632, 29935, 21004, 6693, 10948, 26840, 13848, 4726, 14589, 10590, 25854, 2520, 8870, 11839, 1600, 21570, 21564, 21705, 11099, 8244, 29501, 13311, 15701, 19108, 15094, 15976, 4167, 21301, 18421, 22634, 6270, 20342, 25554, 26263, 10278, 22410, 25822, 26895, 10143, 26454, 20828, 12434, 4471, 15262, 14545, 9846, 9661, 9223, 25623, 19862, 11147, 10353, 4023, 19837, 25896, 16814, 8357, 28545, 947, 22328, 18761, 27218, 797, 5564, 13385, 7595, 17877, 4835, 29630, 6276, 26385, 27139, 24871, 25685, 23355, 28228, 10965, 9048, 13846, 3608, 29563, 26516, 16714, 18221, 20638, 20634, 6520, 12724, 6875, 1423, 3387, 24019, 6590, 21642, 18933, 21671, 22751, 10661, 25598, 27312, 18281, 1230, 22341, 17561, 17144, 28037, 675, 5225, 23752, 5056, 6221, 13713, 13593, 20021, 28249, 11158, 28346, 553, 29085, 22652, 2

In [40]:
train_dataset = CustomDataset(train_image_path, df_train, rand_list_train, data_transforms['train'])
test_dataset = CustomDataset(test_image_path, df_test, rand_list_test, data_transforms['test'])

In [41]:
train_dataset[0]

(tensor([[[-1.3130, -1.3130, -1.2788,  ...,  1.5982,  1.6153,  1.6324],
          [-1.3302, -1.3130, -1.2788,  ...,  1.5810,  1.5982,  1.6153],
          [-1.3302, -1.3302, -1.2959,  ...,  1.5468,  1.5982,  1.5982],
          ...,
          [ 1.7694,  1.7694,  1.7694,  ..., -1.4672, -1.4672, -1.4843],
          [ 1.7694,  1.7694,  1.7694,  ..., -1.4843, -1.5357, -1.5357],
          [ 1.7694,  1.7694,  1.7694,  ..., -1.4843, -1.5185, -1.5185]],
 
         [[-2.0357, -2.0357, -2.0357,  ...,  1.7458,  1.7108,  1.7458],
          [-2.0357, -2.0357, -2.0357,  ...,  1.7108,  1.7108,  1.7108],
          [-2.0357, -2.0357, -2.0357,  ...,  1.6933,  1.6933,  1.6933],
          ...,
          [ 1.7808,  1.7808,  1.7808,  ..., -2.0357, -2.0357, -2.0357],
          [ 1.7808,  1.7808,  1.7808,  ..., -2.0357, -2.0357, -2.0357],
          [ 1.7808,  1.7808,  1.7808,  ..., -2.0357, -2.0357, -2.0357]],
 
         [[-1.8044, -1.8044, -1.8044,  ...,  1.9080,  1.9080,  1.9080],
          [-1.8044, -1.8044,

In [42]:
if SMALL_DATASET == False:
    image, label = train_dataset[100]
    plt.imshow(image.permute(1, 2, 0))
    print(label)
    print(values[1][label])

In [43]:
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [44]:
next(iter(train_loader))

[tensor([[[[ 2.2489,  2.2489,  2.2489,  ...,  2.2489,  2.2489,  2.2489],
           [ 2.2489,  2.2489,  2.2489,  ...,  2.2489,  2.2489,  2.2489],
           [ 2.2489,  2.2489,  2.2489,  ...,  2.2489,  2.2489,  2.2489],
           ...,
           [ 2.2489,  2.2489,  2.2489,  ...,  0.3309,  0.1426,  1.0673],
           [ 2.2489,  2.2489,  2.2489,  ...,  0.2796,  0.1768,  0.2111],
           [ 2.2489,  2.2489,  2.2489,  ...,  0.1768,  0.1768, -0.1143]],
 
          [[ 2.4286,  2.4286,  2.4286,  ...,  2.4286,  2.4286,  2.4286],
           [ 2.4286,  2.4286,  2.4286,  ...,  2.4286,  2.4286,  2.4286],
           [ 2.4286,  2.4286,  2.4286,  ...,  2.4286,  2.4286,  2.4286],
           ...,
           [ 2.4286,  2.4286,  2.4286,  ..., -1.9832, -1.9132,  0.1527],
           [ 2.4286,  2.4286,  2.4286,  ..., -1.8606, -1.7556, -1.0378],
           [ 2.4286,  2.4286,  2.4286,  ..., -1.8782, -1.7556, -1.7906]],
 
          [[ 2.6400,  2.6400,  2.6400,  ...,  2.6400,  2.6400,  2.6400],
           [ 

In [45]:
dataloaders = {'train': train_loader, 'val': test_loader}
dataset_sizes = {'train': len(train_dataset), 'val': len(test_dataset)}
print(dataset_sizes)

{'train': 800, 'val': 200}


In [46]:
next(iter(dataloaders['train']))

[tensor([[[[ 0.7419,  0.8618,  0.9474,  ...,  0.8447,  0.8447,  0.8447],
           [ 0.6734,  0.8961,  0.9817,  ...,  0.8447,  0.8447,  0.8447],
           [ 0.6906,  0.9474,  1.0673,  ...,  0.8447,  0.8447,  0.8447],
           ...,
           [ 0.8447,  0.8447,  0.8447,  ...,  0.8447,  0.8447,  0.8447],
           [ 0.8447,  0.8447,  0.8447,  ...,  0.8447,  0.8447,  0.8447],
           [ 0.8447,  0.8447,  0.8447,  ...,  0.8447,  0.8447,  0.8447]],
 
          [[ 0.7129,  0.9405,  1.0455,  ...,  2.0434,  2.0434,  2.0434],
           [ 0.6604,  0.9755,  1.0630,  ...,  2.0434,  2.0434,  2.0434],
           [ 0.6429,  1.0455,  1.2031,  ...,  2.0434,  2.0434,  2.0434],
           ...,
           [ 2.0434,  2.0434,  2.0434,  ...,  2.0434,  2.0434,  2.0434],
           [ 2.0434,  2.0434,  2.0434,  ...,  2.0434,  2.0434,  2.0434],
           [ 2.0434,  2.0434,  2.0434,  ...,  2.0434,  2.0434,  2.0434]],
 
          [[-0.4973, -0.5670, -0.5147,  ..., -1.8044, -1.8044, -1.8044],
           [-

In [47]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    # Create a temporary directory to save training checkpoints
    with TemporaryDirectory() as tempdir:
        best_model_params_path = os.path.join(tempdir, 'best_model_params.pt')

        torch.save(model.state_dict(), best_model_params_path)
        best_acc = 0.0

        for epoch in range(num_epochs):
            print(f'Epoch {epoch}/{num_epochs - 1}')
            print('-' * 10)

            # Each epoch has a training and validation phase
            for phase in ['train', 'val']:
                if phase == 'train':
                    model.train()  # Set model to training mode
                else:
                    model.eval()   # Set model to evaluate mode

                running_loss = 0.0
                running_corrects = 0

                # Iterate over data.
                for inputs, labels in dataloaders[phase]:
                    inputs = inputs.to(device)
                    labels = labels.to(device)

                    # zero the parameter gradients
                    optimizer.zero_grad()

                    # forward
                    # track history if only in train
                    with torch.set_grad_enabled(phase == 'train'):
                        outputs = model(inputs)
                        _, preds = torch.max(outputs, 1)
                        loss = criterion(outputs, labels)

                        # backward + optimize only if in training phase
                        if phase == 'train':
                            loss.backward()
                            optimizer.step()

                    # statistics
                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += torch.sum(preds == labels.data)
                if phase == 'train':
                    scheduler.step()

                epoch_loss = running_loss / dataset_sizes[phase]
                epoch_acc = running_corrects.double() / dataset_sizes[phase]

                print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

                # deep copy the model
                if phase == 'val' and epoch_acc > best_acc:
                    best_acc = epoch_acc
                    torch.save(model.state_dict(), best_model_params_path)

            print()

        time_elapsed = time.time() - since
        print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
        print(f'Best val Acc: {best_acc:4f}')

        # load best model weights
        model.load_state_dict(torch.load(best_model_params_path, weights_only=True))
    return model

In [48]:
def visualize_model(model, num_images=6):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['val']):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                ax.set_title(f'predicted: {values[0][preds[j]]}')
                imshow(inputs.cpu().data[j])

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)

In [52]:
model_ft = models.resnet18(weights='IMAGENET1K_V1')
num_ftrs = model_ft.fc.in_features
# Here the size of each output sample is set to 2.
# Alternatively, it can be generalized to ``nn.Linear(num_ftrs, len(class_names))``.
model_ft.fc = nn.Sequential(
    nn.Dropout(0.5),
    nn.Linear(num_ftrs, len(values[0]))
)

model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.01, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.CosineAnnealingLR(optimizer_ft, T_max=10)

In [53]:
for param in model_ft.parameters():
    param.requires_grad = False  # Freeze all layers initially
for param in model_ft.fc.parameters():
    param.requires_grad = True  # Unfreeze only the final layer


In [54]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=5)

Epoch 0/4
----------
train Loss: 2.9522 Acc: 0.2162
val Loss: 2.0977 Acc: 0.2000

Epoch 1/4
----------
train Loss: 2.4152 Acc: 0.2725
val Loss: 2.9997 Acc: 0.2300

Epoch 2/4
----------
train Loss: 2.6527 Acc: 0.2687
val Loss: 2.3263 Acc: 0.2850

Epoch 3/4
----------
train Loss: 2.6435 Acc: 0.3050
val Loss: 3.2492 Acc: 0.2100

Epoch 4/4
----------


KeyboardInterrupt: 